In [1]:
# 400 meter coordinate accuracy
# 1923 earliest year
import pandas as pd
import numpy as np
import json
import glob
import os
import geopandas as gpd
from dask import dataframe as dd
import dask_geopandas
from shapely import wkt
import vaex
import time


pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [2]:
# read in gbif_clean
df_path = input("Enter the file path: ")

df = dd.read_parquet(df_path)

In [ ]:
# read in eco_map.geojson
map_path = input("Enter the file path: ")

map = gpd.read_file(map_path)

In [ ]:
df

In [ ]:
map.head()

In [ ]:
map.crs

In [4]:
# convert to dask_geopandas df
df_gpd = dask_geopandas.from_dask_dataframe(df)

In [6]:
df_gpd = df_gpd.set_geometry(
    dask_geopandas.points_from_xy(df_gpd, "decimalLongitude", "decimalLatitude")
)

In [9]:
df_gpd = df_gpd.drop(["decimalLatitude", "decimalLongitude"], axis=1)

In [11]:
df_gpd = df_gpd.set_crs("EPSG:4326")

# if get stuck might be useful at some point
# df_gpd.drop(df_gpd[df_gpd["geometry"].is_empty].index, inplace=True)

# df_gpd["geometry"] = df_gpd["geometry"].apply(wkt.loads)
# df_gpd = gpd.GeoDataFrame(df_gpd, crs="EPSG:4326")


In [ ]:
df_gpd.crs

In [13]:
# only run if dataframe will fit in memory
# df_gpd = df_gpd.compute()

# df_gpd.head()

In [15]:
joined = df_gpd.sjoin(map, predicate="within")

ValueError: 'right_df' should be GeoDataFrame, got <class 'type'>